In [3]:

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

endpoint = "https://ai-mutazaaieastus2054278078742.openai.azure.com/"
model_name = "o3-mini"
deployment = "o3-mini"
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://management.core.windows.net")
#"https://cognitiveservices.azure.com/.default")
api_version = "2024-12-01-preview"



In [4]:
import os
import requests
import json
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
# Azure Management API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.CognitiveServices/modelCapacities"

# Query parameters
params = {
    "api-version": "2024-10-01",
    "modelFormat": "OpenAI",
    "modelName": "o3-mini",
    "modelVersion": "2025-01-31"
}

params = {
    "api-version": "2024-10-01",
    "modelFormat": "OpenAI",
    "modelName": "gpt-4.1",
    "modelVersion": "2025-04-14"
}

# You'll need to authenticate with Azure
# This typically requires an access token from Azure AD
headers = {
    "Authorization": "Bearer " + token_provider(),
    "Content-Type": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Print the response
print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    # print(json.dumps(response.json(), indent=4))
    print("SUCCESS")
else:
    print(f"Error: {response.text}")

Status Code: 200
SUCCESS


In [5]:
import pandas as pd

# Get the JSON data from the response
data = response.json()

# Extract the value list from the JSON response
values = data.get('value', [])

# Create a DataFrame from the values list
if values:
    # Flatten the nested JSON structure if needed
    df = pd.json_normalize(values)
    # Display the DataFrame as a table
    # display(df)
else:
    print("No values found in the response")

In [6]:
df.columns

Index(['id', 'type', 'name', 'location', 'properties.model.format',
       'properties.model.name', 'properties.model.version',
       'properties.skuName', 'properties.availableCapacity',
       'properties.availableFinetuneCapacity'],
      dtype='object')

In [7]:
# Display selected columns from the dataframe
display(df[['name', 'location', 'properties.model.name', 'properties.model.version',
           'properties.skuName', 'properties.availableCapacity']])

,name,location,properties.model.name,properties.model.version,properties.skuName,properties.availableCapacity
0,GlobalProvisionedManaged,australiaeast,gpt-4.1,2025-04-14,GlobalProvisionedManaged,0
1,GlobalProvisionedManaged,brazilsouth,gpt-4.1,2025-04-14,GlobalProvisionedManaged,0
2,DataZoneStandard,westus,gpt-4.1,2025-04-14,DataZoneStandard,300
3,ProvisionedManaged,westus,gpt-4.1,2025-04-14,ProvisionedManaged,1000
4,DataZoneProvisionedManaged,westus,gpt-4.1,2025-04-14,DataZoneProvisionedManaged,0
...,...,...,...,...,...,...
63,DataZoneProvisionedManaged,italynorth,gpt-4.1,2025-04-14,DataZoneProvisionedManaged,0
64,GlobalProvisionedManaged,italynorth,gpt-4.1,2025-04-14,GlobalProvisionedManaged,0
65,DataZoneStandard,spaincentral,gpt-4.1,2025-04-14,DataZoneStandard,300
66,DataZoneProvisionedManaged,spaincentral,gpt-4.1,2025-04-14,DataZoneProvisionedManaged,0


In [ ]:
from IPython.display import display, HTML

# Filter dataframe for rows containing 'ProvisionedManaged' in the SKU name
provisioned_managed_df = df[df['properties.skuName'].str.contains('ProvisionedManaged')]

# Get total count
pm_count = len(provisioned_managed_df)

# Create a summary by location
location_summary = provisioned_managed_df.groupby(['location', 'properties.skuName']).agg({
    'properties.availableCapacity': 'sum'
}).reset_index()

# Sort by available capacity in descending order
location_summary = location_summary.sort_values('properties.availableCapacity', ascending=False)

# Display the results

display(HTML(f"<h3>ProvisionedManaged SKUs ({pm_count} total)</h3>"))
display(location_summary)

# Also display the original filtered dataframe with selected columns
display(HTML("<h4>Detailed view:</h4>"))
display(provisioned_managed_df[['location', 'properties.skuName',
                              'properties.availableCapacity', 'properties.model.name']])

,location,properties.skuName,properties.availableCapacity
46,westus3,ProvisionedManaged,1000
43,westus,ProvisionedManaged,1000
28,southcentralus,ProvisionedManaged,1000
21,northcentralus,ProvisionedManaged,550
6,eastus,ProvisionedManaged,550
9,eastus2,ProvisionedManaged,550
1,brazilsouth,GlobalProvisionedManaged,0
5,eastus,GlobalProvisionedManaged,0
7,eastus2,DataZoneProvisionedManaged,0
8,eastus2,GlobalProvisionedManaged,0


,location,properties.skuName,properties.availableCapacity,properties.model.name
0,australiaeast,GlobalProvisionedManaged,0,gpt-4.1
1,brazilsouth,GlobalProvisionedManaged,0,gpt-4.1
3,westus,ProvisionedManaged,1000,gpt-4.1
4,westus,DataZoneProvisionedManaged,0,gpt-4.1
5,westus,GlobalProvisionedManaged,0,gpt-4.1
7,westeurope,DataZoneProvisionedManaged,0,gpt-4.1
8,westeurope,GlobalProvisionedManaged,0,gpt-4.1
9,southeastasia,GlobalProvisionedManaged,0,gpt-4.1
12,southcentralus,ProvisionedManaged,1000,gpt-4.1
13,southcentralus,DataZoneProvisionedManaged,0,gpt-4.1


In [6]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Get unique locations from the dataframe
locations = sorted(df['location'].unique())

# Create a dropdown widget
location_dropdown = widgets.Dropdown(
    options=locations,
    description='Location:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Create a function to filter the data
def filter_by_location(location):
    filtered_df = df[df['location'] == location]
    display(HTML(f"<h3>Model capacity in {location}</h3>"))
    display(HTML("<h4>Model Name: o3-mini</h4>"))
    display(filtered_df[['properties.model.name','name', 'properties.skuName', 'properties.availableCapacity']])

# Create an interactive output
output = widgets.Output()

# Define what happens when the dropdown value changes
def on_location_change(change):
    with output:
        output.clear_output()
        filter_by_location(change.new)

location_dropdown.observe(on_location_change, names='value')

# Display the widget and initial output
display(location_dropdown)
display(output)

# Show initial data for the first location
with output:
    filter_by_location(locations[0])

Dropdown(description='Location:', layout=Layout(width='300px'), options=('australiaeast', 'brazilsouth', 'cana…

Output()